In [3]:
import pickle
import json
with open('./cid_set.pkl', 'rb') as f:
    cid_set = pickle.load(f)

with open('./chemical_nbr2d_.json', 'r') as f:
    chemical_nbr2d = json.load(f)

with open('./chemical_nbr3d_.json', 'r') as f:
    chemical_nbr3d = json.load(f)


In [4]:
len(cid_set)

62174

In [5]:
import requests
import re
from collections import defaultdict
from tqdm import tqdm

nbr_2d = defaultdict(list)
nbr_3d = defaultdict(list)

i = 0
for cid in tqdm(cid_set):
    i+=1
    if i % 10000 == 0:
        with open('./chemical_nbr2d.json', 'w') as f:
            json.dump(nbr_2d, f, indent=6)

        with open('./chemical_nbr3d.json', 'w') as f:
            json.dump(nbr_3d, f, indent=6)

    if cid in chemical_nbr2d:
        nbr_2d[cid] = chemical_nbr2d[cid]
    if cid in chemical_nbr3d:
        nbr_3d[cid] = chemical_nbr3d[cid]
    
    if cid in nbr_2d and cid in nbr_3d:
        continue
    # Define the API endpoint URL
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/rdf/compound/{cid}/nbr.html"

    # Query the API and parse the response
    response = requests.get(url)
    lines = response.text

    # Extract the CIDs for 2D similarity
    pattern_2d = r'similar to by PubChem 2D similarity algorithm.*?CID(\d+).*?CID(\d+).*?CID(\d+).*?CID(\d+).*?CID(\d+)'
    matches_2d = re.search(pattern_2d, lines, flags=re.DOTALL)
    if matches_2d:
        cids_2d = matches_2d.groups()
        top_2d = []
        for c in cids_2d:
            if c not in top_2d and c != cid:
                nbr_2d[cid].append(c)
            if len(nbr_2d[cid]) == 3:
                break

    # Extract the CIDs for 3D similarity
    pattern_3d = r'similar to by PubChem 3D similarity algorithm.*?CID(\d+).*?CID(\d+).*?CID(\d+).*?CID(\d+).*?CID(\d+)'
    matches_3d = re.search(pattern_3d, lines, flags=re.DOTALL)
    if matches_3d:
        cids_3d = matches_3d.groups()
        top_3d = []
        for c in cids_3d:
            if c not in top_3d and c != cid:
                nbr_3d[cid].append(c)
            if len(nbr_3d[cid]) == 3:
                break


100%|██████████| 62174/62174 [49:53<00:00, 20.77it/s]  


In [ ]:
import json

with open('./chemical_nbr2d.json', 'w') as f:
    json.dump(nbr_2d, f, indent=6)

with open('./chemical_nbr3d.json', 'w') as f:
    json.dump(nbr_3d, f, indent=6)
